</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_groq_response, test_cases=test_cases) # setting manual test cases
# node = EvolNode(mp, None, None, get_response=get_groq_response) # automatic generation of test cases 


node.evolve("i1", replace=True, max_attempts=3, num_runs=2) # Roll the dice 5 times, record the best one

print("Inspection on the generated code: \n", node.code)

import time 
time.sleep(3)

input_dict = test_inputs[0]
output_dict = node(input_dict) # Could still have error due to unsuccessful parsing of output
print("Output from the code: \n", output_dict)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Could not load vllm class, check CUDA support and GPU RAM size


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating fitness: 100%|██████████| 4/4 [00:02<00:00,  1.39it/s]


--- Replacing with new node
 - Attempt 1 failed. Fitness: 0.75. Error:  Function runs with success rate: 75.0%, runs correctly with rate: 75.0%
--- Compiled 3 out of 4 test cases
--- Passed 3 out of 4 test cases
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)
Parse Response Failed...
 - Attempt 2 failed. Fitness: 0.00. Error: 


Evaluating fitness: 100%|██████████| 4/4 [00:02<00:00,  1.70it/s]


 - Attempt 3 failed. Fitness: 0.50. Error:  Function runs with success rate: 50.0%, runs correctly with rate: 50.0%
--- Compiled 2 out of 4 test cases
--- Passed 2 out of 4 test cases
Input: {'name': 'Dilireba'}, Output is missing or of wrong type, Expected: {'age': 32}
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)--- Calling Prompt Function Error:
Failed to parse LLM response: No JSON structure found in the provided text.
Evolution failed after 3 attempts.
Inspection on the generated code: 
 def generate_prompt(name):
    """
    Generate a prompt to guide an AI in getting the age of a celebrity.

    Args:
        name (str): The name of the celebrity.

    Returns:
        str: A string containing the final prompt for the AI.
    """
    prompt = f'G

In [1]:
# Population building phase ... 
from methods.llm import get_groq_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_groq_response, test_cases=test_cases, max_attempts=3, num_eval_runs=1)


Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
node_population = [{'reasoning': "To generate a prompt for an AI to find the age of a celebrity, I will incorporate a natural language approach, leveraging the search engine's ability to provide relevant information, while also specifying the required output format in the prompt for clarity.",
  'code': 'def generate_prompt(name):\n    """\n    Generate a prompt to guide an AI in finding the age of a celebrity.\n\n    Parameters:\n    name (str): The name of the celebrity.\n\n    Returns:\n    str: A string containing the final prompt for the AI.\n    """\n    prompt = (\n        f"Given the input \'{name}\', please provide a JSON-style response with the following structure: "\n        )\n    prompt += (\n        "{\'age\': int(<age>)}}, where <age> is the age of the celebrity in years."\n        )\n    prompt += (\n        \' The response should be based on the latest available information from top search results.\'\n        )\n    return prompt\n',
  'fitness': 1.0},
 {'reasoning': 'To calculate the age of a celebrity, we need to first search for their birth date or age online and then perform date arithmetic to find their current age. We can utilize this reasoning by using the search engine to find the relevant information and then utilizing AI to process the information and calculate the age.',
  'code': 'def generate_prompt(name: str) ->str:\n    """\n    Generates a prompt to guide an AI in calculating the age of a celebrity.\n    \n    Args:\n    name (str): The name of the celebrity.\n    \n    Returns:\n    str: A string containing the final prompt for the AI.\n    """\n    prompt = ("Given the name \'" + name +\n        "\', use the search engine to find the birth date of " + name +\n        \' and calculate their current age.\')\n    return (\'Search google for result \' + prompt +\n        ", format the output as a JSON-style dictionary: {\'age\': int(...)}")\n',
  'fitness': 1.0}] # to replace with loading function

In [3]:
# node_population = [] # build up some population first (so we need a chain of evolution methods 'strategy')
node_population = evo._get_offspring("m1", node_population)

TypeError: MetaPrompt._get_prompt_m1() missing 1 required positional argument: 'indiv'

In [3]:
node_population

[{'reasoning': "To generate a prompt for an AI to find the age of a celebrity, I will incorporate a natural language approach, leveraging the search engine's ability to provide relevant information, while also specifying the required output format in the prompt for clarity.",
  'code': 'def generate_prompt(name):\n    """\n    Generate a prompt to guide an AI in finding the age of a celebrity.\n\n    Parameters:\n    name (str): The name of the celebrity.\n\n    Returns:\n    str: A string containing the final prompt for the AI.\n    """\n    prompt = (\n        f"Given the input \'{name}\', please provide a JSON-style response with the following structure: "\n        )\n    prompt += (\n        "{\'age\': int(<age>)}}, where <age> is the age of the celebrity in years."\n        )\n    prompt += (\n        \' The response should be based on the latest available information from top search results.\'\n        )\n    return prompt\n',
  'fitness': 1.0},
 {'reasoning': 'To calculate the a

In [3]:
# len(node_population)

evo._get_offspring("m1", node_population)

AttributeError: 'Evolution' object has no attribute 'm'

In [16]:
# len(node_population)

code = node_population[1]['code']
print(code)

def generate_prompt(name):
    """
    Generate a prompt to guide an AI in completing the task of finding the age of a celebrity.

    Parameters:
    name (str): The name of the celebrity.

    Returns:
    str: A string containing the final prompt for the AI.
    """
    prompt = (
        f"Please search for {name} and their age/biography on the internet, and return a JSON-style response with an output dictionary of the format {{'age': int(age)}}"
        )
    return prompt



In [12]:
evo.evol._evaluate_fitness(test_cases=test_cases, code=code) # error in evaluation metric ? (need debugging, prediction is 31, but GT is 32, yet it's judged to be correct ??)
# evo.evol.call_prompt_function(code = code, test_input = test_cases[0][0])

Evaluating fitness: 100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


(1.0,
 ' Function runs with success rate: 100.0%, runs correctly with rate: 100.0%\n--- Compiled 2 out of 2 test cases\n--- Passed 2 out of 2 test cases\n\nError Message:\n')

In [10]:
test_cases[0][0]

{'name': 'Dilireba'}

In [24]:
# Obviously, I need a 'reflective and fix shit-up evolution strategy here, too ...'
# sth like 'what is the problem with the code given referrable sub-functions 'xxxxxxx'?'


In [9]:
test_cases

[({'name': 'Dilireba'}, {'age': 32}), ({'name': 'ChengXiao'}, {'age': 26})]

In [8]:
from methods.population import Evolution

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_groq_response, test_cases=test_cases, max_attempts=3, num_eval_runs=1)


Generating test cases: 5case [00:01,  2.61case/s]             

--- Generated 5 test cases


#### Direct Loading Node

In [6]:
from methods.evolnode import EvolNode 
from methods.llm import get_groq_response

node = EvolNode.load("get_celeb_age", get_response=get_groq_response)
# node = EvolNode.load("search_google")
input_dict = {"name": "Dilireba"}
node(input_dict)

{'age': 32}

In [7]:
print(node.code)

from datetime import datetime
import http.client
import json
import os
from typing import Dict, Any


def _search_google(query: str) ->Dict[str, Any]:
    """
    Use Serper API to search Google for information
    
    Args:
        query (str): The search query
    
    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection('google.serper.dev')
    payload = json.dumps({'q': query})
    headers = {'X-API-KEY': os.environ['SERPER_API_KEY'], 'Content-Type':
        'application/json'}
    try:
        conn.request('POST', '/search', payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode('utf-8'))
    except Exception as e:
        print(f'Error occurred during API request: {str(e)}')
        return {}
    finally:
        conn.close()


def search_google(query: str) ->str:
    """ 
    Input query, return search result string from Google
    """
    result = _search_go

#### Alignment-Check for Evaluation

In [14]:
from methods.llm import get_groq_response 

# Evaluation base on alignment check
node.get_response = get_groq_response 
fitness, issue_summary = node._evaluate_fitness(max_tries=1, num_runs=1)


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>

</div>
<div align="center">
  <img src="img/Planning-node.png" width="400" alt="Planning Node Task Decomposition">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.llm import get_claude_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text

# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])

# Step 1: Generate Pseudo-Code for SubTask Decomposition
prompt = mp._get_pseudo_code_prompt() # Pseudo-Code Prompt (Non-implemented functional)
response = get_claude_response(prompt) # Use Strong LLM to build up pseudo-code
code = extract_python_code(response) # Extract Python Code from response 

# Step 2: Generate Planning DAG: Multiple Nodes 
graph_prompt = mp._get_plan_graph_prompt(code) 
plan_response = get_claude_response(graph_prompt)
plan_dict = extract_json_from_text(plan_response)

visualize_plan_dict(plan_dict)

# Step 3: Spawn Multiple Sub-Nodes and Evolve them

# Step 4: Slot Sub Node into context for parent node re-write 

Could not load vllm class, check CUDA support and GPU RAM size


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 207.111084ms
success: successfully compiled d2_output/plan_graph_dag.d2 to d2_output/plan_graph_dag.png in 207.980416ms


In [1]:
# Ok we could load this thing now ..

from methods.eoh_evolution import PlanNode 
from methods.llm import get_groq_response 

plan_node = PlanNode.load("get_celeb_age", get_response=get_groq_response)

Could not load vllm class, check CUDA support and GPU RAM size


In [5]:
plan_dict = plan_node.plan_dict

In [6]:
prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

Generating test cases: 5case [00:05,  1.20s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:09,  1.99s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:06,  1.20s/case]             


...

--- Generated 3 test cases


Generating test cases: 5case [00:06,  1.28s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:10,  2.19s/case]             

--- Generated 5 test cases


In [17]:
# Chicken n Egg problem: - how can you evolve a function which you can't evaluate? 
code_nodes["scrape_celeb_data"].test_cases # probably not fair to expect test-cases for this one to be grounded ... 

[({'name': 'Tom Hanks'}, {'data': '67'}),
 ({'name': 'Emma Watson'}, {'data': '33'}),
 ({'name': 'Leonardo da Vinci'}, {'data': 'Deceased'})]

In [18]:
scrape_node = code_nodes["scrape_celeb_data"]

scrape_node.evolve("i1", replace=True)

Evaluating fitness: 100%|██████████| 15/15 [00:36<00:00,  2.43s/it]


 - Attempt 1 failed. Fitness: 0.00. Error: --- Compiled 10 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given I

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 44906.90it/s]


 - Attempt 2 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 31583.61it/s]


 - Attempt 3 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 7772.03it/s]


 - Attempt 4 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness:   0%|          | 0/15 [01:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Code Node evolution is bad now, parsing seems to be problematic



In [20]:
code = scrape_node.code
print(code)

defined structured data to locate the celebrity based on their name and then retrieving their corresponding age.}

```python
def scrape_celeb_data(name: str) -> str:
    # Predefined mock data to simulate web data
    celeb_data = {
        "Taylor Swift": 33,
        "Leonardo DiCaprio": 48,
        "Beyoncé": 42,
        "Chris Evans": 42,
        "Scarlett Johansson": 38
    }
    
    # Attempt to find the celebrity's age
    age = celeb_data.get(name)

    # If age not found, return a message
    if age is None:
        return f"Age for {name} not found."
    
    # Return the age as a string
    return f"{name} is {age} years old."


In [10]:
# Evolve Sub-Nodes (hope is not too slow here ...)
# 1. Code success -> end | else -> Prompt | pick best -> return node for specific sub-task

# prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

In [24]:
# visualize_plan_dict(plan_dict)

plan_dict # Ignore the mode for now, let's use both mode and pick the best performing ones ... 

plan_dict["nodes"]

# Basic Idea: 
# - Build node libraries is the better way to describe this 



[{'task': 'Search for celebrity in database',
  'name': 'search_celebrity_database',
  'inputs': ['name'],
  'input_types': ['str'],
  'outputs': ['celeb_info'],
  'output_types': ['dict'],
  'target': 'Retrieve celebrity information',
  'mode': 'CODE'},
 {'task': 'Extract birth date from celebrity info',
  'name': 'extract_birth_date',
  'inputs': ['celeb_info'],
  'input_types': ['dict'],
  'outputs': ['birth_date'],
  'output_types': ['date'],
  'target': "Get celebrity's birth date",
  'mode': 'CODE'},
 {'task': 'Get current date',
  'name': 'get_current_date',
  'inputs': [],
  'input_types': [],
  'outputs': ['current_date'],
  'output_types': ['date'],
  'target': 'Obtain current date for age calculation',
  'mode': 'CODE'},
 {'task': 'Calculate age based on birth date and current date',
  'name': 'calculate_age',
  'inputs': ['birth_date', 'current_date'],
  'input_types': ['date', 'date'],
  'outputs': ['age'],
  'output_types': ['int'],
  'target': "Determine celebrity's curr